In [4]:
# !pip3 install wordcloud
# !pip3 install polyglot
# !pip3 install pyicu
# !pip3 install pycld2
# !pip3 install morfessor
# !pip3 install polyglot
# !pip3 install fuzzywuzzy

In [5]:
import os
import pandas as pd
import sys
import numpy as np 
import spacy
import nltk
from nltk import word_tokenize, sent_tokenize, RegexpParser, tree
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz

from tqdm import tqdm_notebook, tqdm
from collections import Counter
import re
import operator
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import string 

## https://markhneedham.com/blog/2017/11/28/python-polyglot-modulenotfounderror-no-module-named-icu/
from polyglot.detect import Detector

tqdm.pandas()

/Users/felisialoukou/.virtualenvs/corona/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/felisialoukou/.virtualenvs/corona/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Load in the survey data

In [6]:
DATA_DIR = "../../data"

survey_filename = os.path.join(DATA_DIR, "joined_uis_all_of_march.csv")
df = pd.read_csv(survey_filename)

/Users/felisialoukou/.virtualenvs/corona/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
### Some row duplication present

In [8]:
print(df.shape, df.intents_clientID.nunique(), df.primary_key.nunique(), df.session_id.nunique())

print(df.columns)

df[df.session_id.isna()].shape

(55198, 58) 24792 35416 43458
Index(['primary_key', 'intents_clientID', 'fullVisitorId', 'visitId',
       'ga_visitNum', 'Started', 'Ended', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6',
       'Q7', 'Q8', 'session_id', 'dayofweek', 'isWeekend', 'hour', 'country',
       'country_grouping', 'UK_region', 'UK_metro_area', 'channelGrouping',
       'deviceCategory', 'total_seconds_in_session_across_days',
       'total_pageviews_in_session_across_days', 'finding_count',
       'updates_and_alerts_count', 'news_count', 'decisions_count',
       'speeches_and_statements_count', 'transactions_count',
       'regulation_count', 'guidance_count', 'business_support_count',
       'policy_count', 'consultations_count', 'research_count',
       'statistics_count', 'transparency_data_count',
       'freedom_of_information_releases_count', 'incidents_count',
       'done_page_flag', 'count_client_error', 'count_server_error',
       'ga_visit_start_timestamp', 'ga_visit_end_timestamp',
       'intents_start

(11042, 58)

In [9]:
df.drop_duplicates("primary_key", inplace = True)
df.reset_index(inplace=True, drop=True)

## Functions for sentence tokenization, part of speech tagging, PII placeholder stripping, ngram computation

In [10]:
nlp = spacy.load("en_core_web_sm")

pii_filtered = ["DATE_OF_BIRTH", "EMAIL_ADDRESS", "PASSPORT", "PERSON_NAME", 
                "PHONE_NUMBER", "STREET_ADDRESS", "UK_NATIONAL_INSURANCE_NUMBER", "UK_PASSPORT"]
pii_regex = "|".join([f"\\[{p}\\]" for p in pii_filtered])
pii_regex

'\\[DATE_OF_BIRTH\\]|\\[EMAIL_ADDRESS\\]|\\[PASSPORT\\]|\\[PERSON_NAME\\]|\\[PHONE_NUMBER\\]|\\[STREET_ADDRESS\\]|\\[UK_NATIONAL_INSURANCE_NUMBER\\]|\\[UK_PASSPORT\\]'

In [11]:
stop_words = list(stopwords.words('english'))
punctuation = list(string.punctuation) + ['’']
token_blacklist = stop_words + punctuation + pii_filtered

def split_sentences(comment):
    return nltk.sent_tokenize(comment)

def remove_stopwords_punctation(sentences):
    return [[(t[0], t[1], t[2]) for t in sent if t[0].lower() not in token_blacklist] for sent in sentences]

def replace_pii_regex(text):
    return re.sub(pii_regex, "", text)

def compute_ngrams(processed_comment, n, stemming=False, filtering=False):
    # processed_comment = part_of_speech_tag(comment)
    if filtering:
        processed_comment = remove_stopwords_punctation(processed_comment)
    index = 2 if stemming else 0
    tokens = [token[index] for sent in processed_comment for token in sent]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram).lower() for ngram in ngrams]


def part_of_speech_tag(comment):
    sentences = split_sentences(comment)
    return [[(token.text, token.tag_, token.lemma_) for token in nlp(sentence)] for sentence in sentences]

In [12]:
t = "This is a test with punctuation’. this is another sentence."
processed_t = part_of_speech_tag(t)
compute_ngrams(processed_t, 2, stemming=False, filtering=True)

['test punctuation', 'punctuation another', 'another sentence']

## Detect feedback language
There is a bit of foreign language spam in some responses, detect non (primarily) english comments and drop

In [13]:
def detect_language(text):
    if text!="-":
        try:
            langs = {language.confidence:language.code for language in Detector(text, quiet=True).languages}
            return langs[max(langs.keys())]
        except:
            return f"[ERROR] {text}"
    return "-"

In [14]:
df['Q3_pii_removed'] = df['Q3'].progress_map(replace_pii_regex)
df = df[(df.Q3_pii_removed.str.len()<4000)]
df['language'] = df['Q3_pii_removed'].progress_map(detect_language)

  0%|          | 0/35393 [00:00<?, ?it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the langua

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

...

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 68%|██████▊   | 24118/35393 [00:02<00:00, 11378.78it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to de

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

In [15]:
lang_dist = df['language'].value_counts().to_dict()
print(f"Number of unique languages: {len(lang_dist)}")
print(f"English: {round((lang_dist['en']*100)/sum(lang_dist.values()), 2)}%")
print(f"-: {round((lang_dist['-']*100)/sum(lang_dist.values()), 2)}%")
list(lang_dist.items())[0:10]

Number of unique languages: 71
English: 90.98%
-: 7.84%


[('en', 32202),
 ('-', 2775),
 ('un', 62),
 ('xh', 49),
 ('da', 48),
 ('gd', 19),
 ('sco', 13),
 ('mg', 12),
 ('sv', 12),
 ('it', 12)]

In [16]:
df['is_en'] = df['language'].isin(["en", "un", "-", "sco"])

### Part of speech tag

In [17]:
df['pos_tag'] = df[['Q3_pii_removed', 'is_en']].progress_apply(lambda x: part_of_speech_tag(x[0]) 
                                                     if x[1] else [], axis=1)
df['lemmas'] = df['pos_tag'].progress_map(lambda x: [token[2] for sent in x for token in sent])

df['words'] = df['pos_tag'].progress_map(lambda x: [token[0] for sent in x for token in sent])

df.to_csv(os.path.join(DATA_DIR, "joined_uis_all_of_march_lang_pos.csv"), index=False)

100%|██████████| 35393/35393 [00:00<00:00, 177974.21it/s]


## Extract noun and verb phrases

In [18]:
part_of_speech_tag(df.Q3_pii_removed.iloc[0])

[[('Searching', 'VBG', 'search'),
  ('for', 'IN', 'for'),
  ('shopping', 'NN', 'shopping'),
  ('advice', 'NN', 'advice'),
  ('for', 'IN', 'for'),
  ('over', 'IN', 'over'),
  ('70', 'CD', '70'),
  ("'s", 'POS', "'s"),
  ('with', 'IN', 'with'),
  ('cancer', 'NN', 'cancer')]]

In [19]:
grammar = r"""
    cc:
    {<CC>}
    pronoun:
    {<DT><IN><PRP>}
    {<IN>?<PRP>}
    noun_verb:
    {<IN>?<JJ.*>*<NN.*>+<HYPH>?<VBD|VBN|VBG><NN.*>*}
    verb:
    {<TO><VB><IN|RP><WRB>}
    {<IN><EX><VB.*>}
    {<RB><TO><VB.*>+}
    {<TO>?<VB.*><IN|WDT|WP>}
    {<WP><VB.*>}
    {<VB.*><RB><VB.*>*}
    {<WDT>?<TO>?<MD|VB.*>?<RB>?<TO|IN>?<V.*>+<CC>?<V.*>*<IN|RP>?<IN>?}
    {<MD><RB>*<VB.*>*}
    {<VB.*><IN|TO><IN>?}
    {<TO><VB.*><IN>+}
    prep_noun:
    {<IN>+<PRP\$>?<NN><CD>?}
    {<IN><CD><.*>}
    {<RP>?<IN>+<JJ.*>*<NN.*>+}
    {<IN><DT><NN.*><JJ.*>*<NN><HYPH>?<NN>}
    {<IN><NN.*>(<HYPH>?<NN.*>)?}
    {<JJ.*>*<IN><DT>?<NN.*>+<CD>?<NN.*>?}
    {<IN><DT>*<JJ>?<CD>?<NN.*>+<CD>?<NN.*>?}
    noun:
    {<CD><NN.*>}
    {<DT><NN.*>}
    {<JJ.*><NN.*><CD>}
    {<NN.*><CD><JJ.*>?}
    {<JJ.*|NN.*><IN|TO><PRP>}
    {<CD><NN.*><JJ.*>}
    {<RB>*<JJ.*>*<NN.*>*}
    {<DT><JJ.*>*<NN.*>+}
    {<NN.*><CD>?<JJ.*>*<NN.*>*}
    {<IN>+<CD>*<POS>*<IN>*<NN.*>}
    {<IN><PRP\$>?<JJ.*>*<NN.*>}
    {<NN.*><HYPH><NN.*>}
    {<DT>?<CD>?<JJ.*>?<CC>?<JJ.*>?<NN.*>+}
    {<NN.*><HYPH>?<NN.*|JJ.*|VB.*>*}
    {(<NN|NNS>|<NNP|NNPS>)<NNP|NN|NNS|NNPS>+}
    {(<NN|NNS>+|<NNP|NNPS>+)<IN|CC>(<PRP\$|DT><NN|NNS>+|<NNP|NNPS>+)}
    {<JJ|RB|CD>*<NNP|NN|NNS|NNPS>+}
    {<NNP|NN|NNS|NNPS>+}
    adjective:
    {<RB>*<JJ.*><CD>?}
    """

class Chunk:

    def __init__(self, label, tokens, indices):
        self.label = label
        self.tokens = tokens
        self.indices = indices
        self.text = self.text()
        self.lemma = self.lemma()
        self.important_lemma = self.important_lemma()
        self.important_word = self.important_word()

    def text(self):
        return " ".join([w for w,  _ , _  in self.tokens])
    
    def lemma(self):
        return " ".join([l for _,  _ , l  in self.tokens])
    
    def important_word(self):
        return " ".join([w for w,  pos , _  in self.tokens if re.search(r"(NN)|(VB)|(JJ)|(DT)|(RB)|(CD)", pos) ])
    
    def important_lemma(self):
        return " ".join([l for _,  pos , l  in self.tokens if re.search(r"(NN)|(VB)|(JJ)|(DT)|(RB)|(CD)", pos) ])
    
parser = RegexpParser(grammar)

def chunk_text(tagged):
    chunks = parser.parse(tagged)
    index = 0
    segments = []
    for el in chunks:
        if type(el) == tree.Tree:
            chunk = Chunk(el.label(), el.leaves(), list(range(index, index + len(el.leaves()))))
            segments.append(chunk)
            index += len(el.leaves())
        else:
            index += 1
    return segments

def extract_phrase(sentences, merge_inplace=False):
    chunks = []
    for sentence in sentences:
        chunks.append(chunk_text(sentence))
    if merge_inplace:
        return [merge_adjacent_chunks(chunk) for chunk in chunks]
    return chunks  

def merge_adjacent_chunks(chunks):
    merged = []
    previous_label = ""
    for chunk in chunks:
        if chunk.label == previous_label:
            merged[-1] = Chunk(chunk.label, 
                               merged[-1].tokens + chunk.tokens, 
                               merged[-1].indices + chunk.indices)
        else:
            merged.append(chunk)
        previous_label = chunk.label
    return merged

def compute_combinations(sentences, n):
    return [chunks[i:i+n] for chunks in sentences for i in range(len(chunks)-(n-1))]
  

In [20]:
example = df.iloc[7]
example = df[df.Q3.str.contains("age 80")].iloc[0]

print(example.Q3)
print()
print(example.pos_tag)
print()
for sent in extract_phrase(example.pos_tag, True):
    for chunk in sent:
        print("{0:10} {1:20} {2}".format(chunk.label.upper(), chunk.text, chunk.indices))
    print()
    for combo in compute_combinations([sent], 2):
        print(f"{combo[0].text}, {combo[1].text}")
    
#     for combo in compute_combinations([sent], 3):
#         print(f"{combo[0].text}, {combo[1].text}, {combo[2].text}")
    print("=====")

Reporting to inform you that I am in Nairobi visiting. Due to my age 80 I have decided to go back mid May once Coronavirus issue is settled down. 

[[('Reporting', 'VBG', 'report'), ('to', 'TO', 'to'), ('inform', 'VB', 'inform'), ('you', 'PRP', '-PRON-'), ('that', 'IN', 'that'), ('I', 'PRP', '-PRON-'), ('am', 'VBP', 'be'), ('in', 'IN', 'in'), ('Nairobi', 'NNP', 'Nairobi'), ('visiting', 'VBG', 'visit'), ('.', '.', '.')], [('Due', 'IN', 'due'), ('to', 'IN', 'to'), ('my', 'PRP$', '-PRON-'), ('age', 'NN', 'age'), ('80', 'CD', '80'), ('I', 'PRP', '-PRON-'), ('have', 'VBP', 'have'), ('decided', 'VBN', 'decide'), ('to', 'TO', 'to'), ('go', 'VB', 'go'), ('back', 'RB', 'back'), ('mid', 'JJ', 'mid'), ('May', 'NNP', 'May'), ('once', 'IN', 'once'), ('Coronavirus', 'NNP', 'Coronavirus'), ('issue', 'NN', 'issue'), ('is', 'VBZ', 'be'), ('settled', 'VBN', 'settle'), ('down', 'RP', 'down'), ('.', '.', '.')]]

VERB       Reporting to inform  [0, 1, 2]
PRONOUN    you that I           [3, 4, 5]
VERB      

In [21]:
## Compute linguistic pattern combinations

In [22]:
def compute_linguistic_patterns(df_series, n):
    pattern_dictionary = {}

    for vals in tqdm_notebook(df_series.values):
        sents = extract_phrase(vals, True)
        
        for combo in compute_combinations(sents, n):
            key = tuple([c.label for c in combo])
            counter_key =  tuple([c.text.lower() for c in combo])

            if key not in pattern_dictionary.keys():
                pattern_dictionary[key]=Counter()

            pattern_dictionary[key][counter_key]+=1
            
    return pattern_dictionary

In [23]:
pattern_d = compute_linguistic_patterns(df.pos_tag, 2)
pattern_d.keys(), len(pattern_d)

for i, (k,v) in enumerate(sorted(pattern_d.items(), 
                                 key = lambda x: len(x[1]), 
                                 reverse=True), 
                          1):
    print(f"{i}. {' - '.join([ks.upper() for ks in k])} : {len(v)}\n-------------")
    for j, (kk,vv) in enumerate(pattern_d[k].most_common(50), 1):
        print(f"{j}. \'{' '.join([f'[{kks}]' for kks in kk])}\' : {vv}")
    print()
    print("=======\n")

/Users/felisialoukou/.virtualenvs/corona/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



1. VERB - NOUN : 65084
-------------
1. '[need] [help]' : 150
2. '[looking for] [information]' : 130
3. '[to stay at] [home]' : 126
4. '[am] [unable]' : 118
5. '[to find] [information]' : 110
6. '[have] [copd]' : 99
7. '[looking for] [advice]' : 78
8. '[is] [available]' : 75
9. '[is] [due]' : 75
10. '[to get] [information]' : 69
11. '[go to] [work]' : 67
12. '[to get] [help]' : 65
13. '[stay at] [home]' : 64
14. '[to renew] [driving licence]' : 60
15. '[renew driving] [licence]' : 57
16. '[have] [asthma]' : 57
17. '[is not] [clear]' : 55
18. '[to register as] [vulnerable person]' : 53
19. '[have] [no income]' : 52
20. '[is] [impossible]' : 51
21. '[to work from] [home]' : 49
22. '[to get] [advice]' : 49
23. '[am] [disabled]' : 46
24. '[am] [due]' : 46
25. '[was looking for] [information]' : 46
26. '[to be] [able]' : 45
27. '[can not work from] [home]' : 45
28. '[am] [key worker]' : 44
29. '[am] [worried]' : 44
30. '[staying at] [home]' : 42
31. '[to find] [advice]' : 42
32. '[to find 

## Find specific mentions
### Find information

In [24]:
bool(re.search("info|advice|guidance|look|seek|obtain", "i am looking for fo", re.IGNORECASE))

True

In [25]:
str1 = "a delivery slot"
str2 = "online slot"
ratio = fuzz.ratio(str1.lower(), str2.lower())
partial_ratio = fuzz.partial_ratio(str1.lower(), str2.lower())
ratio, partial_ratio

(62, 73)

In [26]:
### Self-identifying am/is/are

In [27]:
identifier = Counter()
for pattern in [("verb", "noun"), ("verb", "prep_noun")]:
    print(pattern)
    for k,v in pattern_d[pattern].items():
        if re.search(r"(^a?((\’|\')?m))$", k[0]):
            identifier[k[1]] +=v
identifier.most_common(100)

('verb', 'noun')
('verb', 'prep_noun')


[('unable', 141),
 ('worried', 75),
 ('disabled', 66),
 ('due', 62),
 ('key worker', 58),
 ('concerned', 44),
 ('vulnerable', 43),
 ('high risk', 41),
 ('eligible', 38),
 ('vulnerable person', 35),
 ('diabetic', 34),
 ('able', 29),
 ('asthmatic', 29),
 ('a carer', 21),
 ('entitled', 19),
 ('single parent', 18),
 ('unsure', 18),
 ('pregnant', 17),
 ('confused', 17),
 ('sure', 16),
 ('self isolating', 15),
 ('aware', 13),
 ('elderly', 13),
 ('scared', 13),
 ('a pensioner', 13),
 ('carer', 12),
 ('desperate', 11),
 ('an agency worker', 11),
 ('about', 11),
 ('full time carer', 11),
 ('ok', 10),
 ('afraid', 10),
 ('fit', 10),
 ('a type diabetic', 10),
 ('unemployed', 9),
 ('interested', 9),
 ('vulnerable adult', 8),
 ('73 years old', 8),
 ('british', 8),
 ('a teacher', 8),
 ('single mum', 8),
 ('sole carer', 8),
 ('sole trader', 8),
 ('at risk', 8),
 ('well', 7),
 ('70 years old', 7),
 ('type 2 diabetic', 7),
 ('75 years old', 7),
 ('homeless', 7),
 ('sick', 7),
 ('a nurse', 7),
 ('wife', 

In [28]:
### Play around with Fuzzywuzzy for imperfect matches (to be aggregated)

In [29]:
str1 = "a delivery slot"
str2 = "online slot"
ratio = fuzz.ratio(str1.lower(), str2.lower())
partial_ratio = fuzz.partial_ratio(str1.lower(), str2.lower())
ratio, partial_ratio

(62, 73)

In [30]:
for identity,_ in identifier.most_common(100):
    for identity2,_ in identifier.most_common(100):
        if identity != identity2:
            pr = fuzz.partial_ratio(identity.lower(), identity2.lower())
            if pr >= 80:
                token = fuzz.token_set_ratio(identity, identity2)
                print(f"{identity}, {identity2} : {pr} [{token}]")

unable, able : 100 [80]
disabled, able : 100 [67]
disabled, disabled person : 100 [100]
key worker, an agency worker : 80 [75]
key worker, a keyworker : 95 [86]
vulnerable, vulnerable person : 100 [100]
vulnerable, able : 100 [57]
vulnerable, vulnerable adult : 100 [100]
vulnerable, extremely vulnerable person : 100 [100]
high risk, high risk person : 100 [100]
vulnerable person, vulnerable : 100 [100]
vulnerable person, able : 100 [38]
vulnerable person, extremely vulnerable person : 100 [100]
vulnerable person, so : 100 [21]
diabetic, a type diabetic : 100 [100]
diabetic, type 2 diabetic : 100 [100]
diabetic, type 1 diabetic : 100 [100]
able, unable : 100 [80]
able, disabled : 100 [67]
able, vulnerable : 100 [57]
able, vulnerable person : 100 [38]
able, vulnerable adult : 100 [40]
able, extremely vulnerable person : 100 [26]
able, disabled person : 100 [42]
asthmatic, with asthma : 80 [70]
a carer, carer : 100 [100]
a carer, full time carer : 86 [83]
a carer, sole carer : 86 [83]
a c

72 years old, 73 years old : 92 [92]
72 years old, 70 years old : 92 [92]
72 years old, 75 years old : 92 [92]
72 years old, 68 years old : 83 [86]
72 years old, 71 years old : 92 [92]
72 years old, 80 years old : 83 [86]
72 years old, 72 years : 100 [100]
72 years old, 77 years old : 92 [92]
72 years old, 66 years old : 83 [86]
72 years old, 85 years old : 83 [86]
72 years old, 79 years old : 92 [92]
72 years old, 76 years : 88 [77]
72 years old, old : 100 [100]
72 years old, 84 years old : 83 [86]
72 years old, 74 years : 88 [77]
a keyworker, key worker : 95 [86]
a keyworker, essential worker : 80 [59]
with asthma, asthmatic : 80 [70]
a supply teacher, a teacher : 89 [100]
84 years old, 73 years old : 83 [86]
84 years old, 70 years old : 83 [86]
84 years old, 75 years old : 83 [86]
84 years old, 68 years old : 96 [92]
84 years old, 71 years old : 83 [86]
84 years old, 80 years old : 92 [92]
84 years old, 77 years old : 83 [86]
84 years old, 66 years old : 83 [86]
84 years old, 85 yea

In [32]:
### Get help

def has_info_req(text):
    if any([bool(re.search("help|assist|support", word, re.IGNORECASE)) for word in text]):
        return "help"
    if any([bool(re.search("info|advice|guidance|look|seek|obtain", word, re.IGNORECASE)) for word in text]):
        return "info"

In [33]:
info_seeking_verbs = {}
for k,v in pattern_d[("verb", "noun")].items():
    if has_info_req([k[1]])!=None and has_info_req([k[1]])=="help":
        print(has_info_req([k[1]]), k, v)
#         info_seeking_verbs[k[0]]
info_seeking_verbs.most_common(20)

help ('can obtain', 'help') 1
help ('is', 'unhelpful') 4
help ("do n't have", 'help') 1
help ('can receive', 'help') 1
help ('need', 'help') 150
help ('to get', 'any help') 3
help ('is', 'no help') 18
help ('was on', 'a life support machine') 1
help ('have got', 'help most') 1
help ('are entitled to', 'any help') 1
help ('longer receive', 'support') 1
help ('to find out', 'help') 29
help ('looking for', 'help') 40
help ('was looking for', 'info help') 1
help ('wanted', 'help') 7
help ('wanted to register for', 'additional support') 1
help ('to see what', 'financial help') 3
help ('to seek', 'financial support') 1
help ('will get', 'any support') 1
help ('will get', 'financial help') 2
help ('to apply for', 'help') 1
help ("'m looking for", 'help') 3
help ('get', 'help') 33
help ('have', 'no support') 2
help ('need', 'help so bad') 1
help ("'s not", 'helpful') 1
help ('needed', 'help') 8
help ('see if', 'any help') 1
help ('needing', 'help') 4
help ('to find out about', 'financial help'

help ('hours.holding on for', '20 minutes each attempt no help line no information') 1
help ('not geting', 'help') 1
help ('rang', 'helpline') 1
help ('typed', 'local support groups') 1
help ('clicking', 'help') 1
help ('found', 'no help so far even citizens advice') 1
help ('have', 'no idea help') 1
help ('have been revised which has been very', 'helpful') 1
help ('would have been extremely', 'helpful') 1
help ('have', 'other care support') 1
help ('will get', 'any help') 6
help ('is completely', 'unhelpful') 1
help ('to see supposed', 'help') 1
help ('wanting to determine', 'helpful official government site') 1
help ('tried to find', 'information help') 1
help ('is', 'no business support') 1
help ('already have', 'a support system') 1
help ('to get', 'clarification help') 1
help ('now have', 'no support') 1
help ('find out if there is', 'any support available') 1
help ('hoping for', 'real thought ideas help') 1
help ('is', 'no support net work') 1
help ('can not get', 'any help') 3
h

help ('have', 'support') 1
help ('went on', 'help') 1
help ('not provide', 'helpful info') 1
help ('get', 'financial assistance') 2
help ('to see what', 'financial support available') 1
help ("is n't clear what", 'help') 1
help ('have not found', 'any help advice') 1
help ('still need', 'any support') 1
help ('looking for', 'assistance') 3
help ('could do with', 'help') 1
help ('now need', 'more help') 1
help ("ca n't get through to", 'the helpline') 1
help ('wanted to find out about', 'support') 1
help ('was trying to find', 'help') 1
help ('to see what', 'help financially') 2
help ('have tried to find out', 'financial support') 1
help ('’s', 'no help whatsoever') 1
help ('to research', 'government support') 2
help ('which is', 'no help') 1
help ('need', 'clear help') 2
help ('seems to be', 'other helpful link') 1
help ('to find to access', 'help') 1
help ('to offer', 'help') 2
help ('to find', 'help elderly inlaws') 1
help ('being', 'vulnerable help') 1
help ('is not', 'enough help')

help ('are worried', 'the governments employee assistance') 1
help ('get', 'help food boxes') 1
help ('to look for', 'self employment help') 1
help ('to phone', 'the warm front helpline 81 year old father') 1
help ('trying to get', 'help vulnerable parents') 1
help ('at told would be on', 'sainsbury helpline') 1
help ('rent what', 'financial help there') 1
help ('could get', 'some help') 1
help ('look for', 'buiness support') 1
help ('can get', 'any money helpe no work') 1
help ('would receive', 'extra help') 1
help ('offered', 'help') 1
help ('has closed what', 'help') 1
help ('wondered what', 'support') 1
help ('stated on', 'sainsbury helpline phone message') 1
help ("'m cleaning", 'income support') 1
help ('is requested', 'support') 1
help ('to ring', 'the helpline') 1
help ('should be given', 'some help') 1
help ('see if get', 'help') 1
help ('back cancelled', 'helpline numbers') 1
help ('are not working', 'airline helplines again unavailable') 1
help ('has', 'no help flights') 1
h

AttributeError: 'dict' object has no attribute 'most_common'

In [34]:
i = 1
rowlist = []
for key in [key for key in pattern_d.keys() ]:
#     print(key)
    for k,v in pattern_d[key].most_common():
        if "delivery" in k[1] or "deliver" in k[0]:
#             rowlist.append({f""})
            print(f"{i}. [{k[0]}] {k[1]} : {v}")
            i+=1
    print()
    i=1

1. [to get] a delivery slot : 13
2. [can not get] a delivery slot : 11
3. [to get] home delivery : 11
4. [can not get] a delivery : 9
5. [to register for] home delivery : 6
6. [to get] a delivery : 5
7. [to get] delivery : 5
8. [can not get] a food delivery : 5
9. [need] home delivery : 5
10. [ca n't get] a delivery slot : 4
11. [are] no delivery slots : 3
12. [to get] online delivery : 3
13. [can not get] home delivery : 3
14. [to get] a home delivery : 3
15. [can get] home delivery : 3
16. [deliver] food : 3
17. [to deliver] food : 3
18. [can not get] delivery : 2
19. [to obtain] a delivery slot : 2
20. [get] a delivery slot : 2
21. [qualified for] home delivery : 2
22. [ca n’t get] a food delivery slot : 2
23. [trying to register for] home delivery : 2
24. [to get] a supermarket delivery : 2
25. [can get] a food delivery : 2
26. [get] delivery slots : 2
27. [register for] home delivery : 2
28. [tried to book] a home delivery slot : 2
29. [have] a delivery : 2
30. [to get] any delive

1. [you] can delivery : 1

1. [are being delivered] to people living : 1
2. [directed] from waitrose site regarding priority shopping delivery slots : 1
3. [are] delivery drivers coming : 1
4. [am] elderly vulnerable customer wishing home delivery : 1
5. [now to get delivered] reason causing : 1
6. [to get] online shopping delivery had : 1
7. [to find out how to get registered] for home delivery services being : 1
8. [to register as] volronable person requiring home delivery service : 1
9. [disabled even] latest delivery was : 1
10. [are] stone yard offering stone delivery : 1
11. [ca nt get.a] delivery morrisons asked : 1
12. [qualify for] food delivery service starting : 1
13. [deliver] to customers allowed : 1
14. [are having] difficulties getting delivery slots : 1
15. [is] high risk asthma diabetes.getting delivery : 1
16. [am having] problems getting food home delivery slots : 1
17. [would like is to have] sainsbury home delivery put : 1
18. [work as] delivery driver delivering c

## Compute triples

In [36]:
pd_triples = compute_linguistic_patterns(df.pos_tag, 3)

for i, (k,v) in enumerate(sorted(pd_triples.items(), 
                                 key = lambda x: len(x[1]), 
                                 reverse=True), 
                          1):
    print(f"{i}. {' - '.join([ks.upper() for ks in k])} : {len(v)}\n-------------")
    for j, (args, counts) in enumerate(pd_triples[k].most_common(10), 1):
        print(f"{j}. {args}: {counts}")
    print()
    print("=======\n")

/Users/felisialoukou/.virtualenvs/corona/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



1. PRONOUN - VERB - NOUN : 29433
-------------
1. ('i', 'am', 'unable'): 75
2. ('i', 'need', 'help'): 64
3. ('i', 'have', 'copd'): 64
4. ('i', 'was looking for', 'information'): 43
5. ('i', 'am', 'due'): 39
6. ('me', 'to stay at', 'home'): 37
7. ('i', 'am', 'disabled'): 37
8. ('it', 'is', 'impossible'): 31
9. ('i', 'have', 'asthma'): 31
10. ('i', 'have', 'no income'): 30


2. VERB - NOUN - PREP_NOUN : 20178
-------------
1. ('to stay at', 'home', 'for 12 weeks'): 10
2. ('need', 'help', 'with shopping'): 5
3. ('to get', 'information', 'on coronavirus'): 4
4. ('is', 'nothing', 'on your website'): 4
5. ('to find out', 'information', 'on coronavirus'): 4
6. ('to change', 'address', 'on my driving'): 4
7. ('looking for', 'information', 'on coronavirus'): 3
8. ('can get', 'help', 'with food'): 3
9. ('is', 'due', 'for renewal'): 3
10. ('to change', 'the address', 'on my driving'): 3


3. NOUN - VERB - NOUN : 19243
-------------
1. ('self', 'isolate for', '12 weeks'): 29
2. ('help', 'is', 'av

## Compute `arg1` - `arg2` co-occurrence db - couples

In [39]:
pattern_db = {}

for vals in tqdm_notebook(df.pos_tag.values):
    sents = extract_phrase(vals, True)
    for combo in compute_combinations(sents, 2):
        key = (combo[0].label, combo[1].label)
        arg1 = combo[0].important_word.lower()
        arg2 = combo[1].text.lower()
        
        if key not in pattern_db.keys():
            pattern_db[key] = {}
        if arg1 not in pattern_db[key].keys():
            pattern_db[key][arg1] = Counter()
            
        pattern_db[key][arg1][arg2]+=1

print(f"There are {len(pattern_db)} possible grammatical combos.")
for i, (k,v) in enumerate(sorted(pattern_db.items(),
                         key = lambda x: len(x[1].values()),
                         reverse= True)[0:15],
                                 1):
    print(k, len(v))

/Users/felisialoukou/.virtualenvs/corona/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



There are 30 possible grammatical combos.
('verb', 'noun') 18393
('noun', 'verb') 13265
('noun', 'prep_noun') 11254
('noun', 'cc') 8031
('noun', 'pronoun') 6238
('verb', 'pronoun') 5183
('prep_noun', 'verb') 4149
('prep_noun', 'noun') 3971
('prep_noun', 'cc') 3102
('prep_noun', 'pronoun') 2272
('verb', 'cc') 2174
('verb', 'prep_noun') 2101
('noun_verb', 'noun') 1961
('noun_verb', 'verb') 1906
('noun_verb', 'prep_noun') 1783


## Wordnet categorization of individual arguments

In [40]:
def get_wordnet_pos(pos):
    if pos.startswith("NN"):
        return wn.NOUN
    if pos.startswith("VB"):
        return wn.VERB
    
def wordnet_category(word, pos):
    wn_pos = get_wordnet_pos(pos)
    if len(word.split(" ")) > 1:
        word = word.split(" ")[-1]
    if len(wn.synsets(word, wn_pos))>0 :
        syn = wn.synsets(word, wn_pos)[0]
        return syn.lexname()
    return "?"

In [41]:
wordnet_category("no one", "NN")

'noun.quantity'

In [42]:
def bulk_compute_categories(argument_list):
    counter = Counter()
    for argument, counts in argument_list.items():
        wordnet_cat = wordnet_category(argument, "NN")
        if "Tops" in wordnet_cat:
            wordnet_cat = f"noun.{argument.lower().split(' ')[-1]}"
        counter[wordnet_cat] += counts
    return counter

In [43]:
key = ('verb', 'noun')
print(f"There are {len(pattern_db[key])} {key[0]}s, accompanied by {key[1]}s.")
for i, (arg1, arg2) in enumerate(sorted(pattern_db[key].items(),
                         key = lambda x: sum(x[1].values()),
                         reverse= True)[:50],
                                 1):
    print(f"{i}. {arg1} :: {sum(arg2.values())} \n-----------")
    print(bulk_compute_categories(arg2).most_common(10))
    for j, (arg2_val, arg2_counts) in enumerate(arg2.most_common(20), 1):
        print(f"{j}. {arg2_val} : {arg2_counts}")
        print(wordnet_category(arg2_val, "NN"))
    print("=======")

There are 18393 verbs, accompanied by nouns.
1. is :: 3617 
-----------
[('?', 1041), ('noun.communication', 377), ('noun.person', 313), ('noun.act', 248), ('noun.attribute', 216), ('noun.state', 199), ('noun.group', 195), ('noun.time', 175), ('noun.cognition', 163), ('noun.location', 158)]
1. available : 79
?
2. due : 76
noun.attribute
3. impossible : 53
noun.cognition
4. vulnerable : 44
?
5. essential : 42
noun.object
6. no information : 41
noun.communication
7. unable : 38
?
8. nothing : 36
noun.quantity
9. safe : 31
noun.artifact
10. high risk : 24
noun.state
11. no help : 22
noun.act
12. key worker : 22
noun.person
13. possible : 22
noun.cognition
14. no advice : 21
noun.communication
15. no mention : 21
noun.communication
16. closed : 20
?
17. open : 20
noun.location
18. pregnant : 20
?
19. unclear : 20
?
20. ok : 19
noun.location
2. have :: 2878 
-----------
[('noun.state', 621), ('?', 397), ('noun.person', 289), ('noun.communication', 262), ('noun.act', 216), ('noun.possession'

[('noun.communication', 111), ('noun.act', 77), ('?', 69), ('noun.person', 65), ('noun.group', 60), ('noun.possession', 30), ('noun.artifact', 28), ('noun.cognition', 21), ('noun.state', 16), ('noun.location', 15)]
1. help : 36
noun.act
2. the government : 16
noun.group
3. the list : 11
noun.communication
4. people : 8
noun.group
5. husband : 7
noun.person
6. the advice : 6
noun.communication
7. the rules : 5
noun.cognition
8. guidance : 5
noun.communication
9. daughter : 5
noun.person
10. a list : 5
noun.communication
11. benefits : 4
noun.possession
12. many : 4
?
13. advice : 4
noun.communication
14. support : 4
noun.act
15. updates : 4
noun.communication
16. flights : 3
noun.group
17. long : 3
?
18. anyone : 3
?
19. measures : 3
noun.act
20. businesses : 3
noun.group
16. do :: 528 
-----------
[('noun.act', 121), ('?', 97), ('noun.person', 55), ('noun.communication', 35), ('noun.state', 34), ('noun.time', 23), ('noun.cognition', 23), ('noun.group', 20), ('noun.possession', 19), ('n

[('?', 35), ('noun.act', 25), ('noun.cognition', 23), ('noun.communication', 23), ('noun.artifact', 21), ('noun.person', 21), ('noun.attribute', 18), ('noun.location', 18), ('noun.group', 17), ('noun.state', 15)]
1. impossible : 11
noun.cognition
2. safe : 11
noun.artifact
3. the point : 5
noun.cognition
4. work : 4
noun.act
5. mot : 4
noun.communication
6. possible : 3
noun.cognition
7. due : 3
noun.attribute
8. a joke : 3
noun.communication
9. ok : 3
noun.location
10. nothing : 3
noun.quantity
11. a lot : 2
noun.quantity
12. the law : 2
noun.group
13. a nightmare : 2
noun.state
14. essential work : 2
noun.act
15. more : 2
noun.person
16. no help : 2
noun.act
17. business : 2
noun.group
18. the help : 2
noun.act
19. coronavirus : 2
?
20. easy : 2
?
32. make :: 247 
-----------
[('noun.communication', 85), ('?', 34), ('noun.act', 30), ('noun.cognition', 20), ('noun.state', 18), ('noun.artifact', 11), ('noun.possession', 9), ('noun.group', 9), ('noun.location', 9), ('noun.attribute', 6)